In [4]:
import pandas as pd
import numpy as np
from numpy import nan as NA

## Pandas NA Handling Methods
- `dropna` filter axis labels based on whether values for each label have missing data, with varying thresholds for how much missing data to tolerate
- `fillna` fill in missing data with some value or use interpolation method such as `fffill` or `bfill`
- `isnull` return boolean values indicating which values are missing / NA
- `notnull` negation of `isnull`

## Filtering out Missing Data

In [5]:
data = pd.Series([1, NA, 3.5, NA, 7])
data.dropna()

#equivalent to data[data.notnull()]


0    1.0
2    3.5
4    7.0
dtype: float64

### dataframes more complex
may want to drop rows or columns that are all NA or only those containing any NAs  
`dropna` by default drops any row containing missing value:


In [15]:
data = pd.DataFrame([[1., 6.5, 3.], [1., NA, NA], [NA,NA,NA], [NA, 6.5, 3.]])

cleaned = data.dropna()

#passing `how='all'` wil only drop rows that are all NA
cleaned = data.dropna(how='all')

#to drop columns in the same way pass axis=1
data[4] = NA
cleaned = data.dropna(axis=1, how='all')

#only want to keep rows containing a certain number of observations
#indicate with `thresh` argument

df = pd.DataFrame(np.random.randn(7,3))
df.iloc[:4, 1] = NA
df.iloc[:2, 2] = NA
#remove any rows with 2 or more NaN observations
df.dropna(thresh=2)

,0,1,2
2,1.113663,NaN,0.837460
3,0.252013,NaN,0.266540
4,-0.043039,-0.492049,-1.238121
5,-1.033481,0.348020,0.698975
6,-0.644540,-0.607313,1.055771


## Filling in missing data
`fillna` is the workhorse function to use  
calling with a contast replaces missing values with that value  
returns new object but can modify existing object in place with parameter `inplace=True`  
`ffill` propogate last valid observation forward to next valid  
`bfill` use next valid observation to fill gap

In [19]:
# replace all NaN with 0
df.fillna(0)

#use dict to replace fill value for each column
df.fillna({1: 0.5, 2: 0})

# pass mean to fill NaN
df.fillna(df.mean())

,0,1,2
0,-0.635107,-0.250447,0.324125
1,-0.080945,-0.250447,0.324125
2,1.113663,-0.250447,0.837460
3,0.252013,-0.250447,0.266540
4,-0.043039,-0.492049,-1.238121
5,-1.033481,0.348020,0.698975
6,-0.644540,-0.607313,1.055771


## Data Transformation
### Removing Duplicates
`data.duplicated()` returns boolean Series indicating whether each row is a duplicate (observed in a previous row) or not  
`data.drop_duplicates()` returns a DF where duplicated array is False

These methods consider all the columns - can specify a subset column to remove any duplicates of just that column  
`data.drop_duplicates(['column_name'])`

Passing `keep='last'` will keep the last observed duplicate while the default drops the last and keeps the first  
`data.drop_duplicates(['column_one', 'column_two'], keep='last')`

### Using a function or mapping
Example: want to add a column indicating type of animal that each food came from

`map` method on a Series accepts a function or dict like object containing a mapping  



In [24]:
## 
data = pd.DataFrame({'food': ['bacon', 'pulled pork', 'bacon', 'Pastrami', 'corned beef', 'Bacon', 'pastrami', 'honey ham', 'nova lox'], 'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})

meat_to_animal = {
    'bacon': 'pig', 'pulled pork': 'pig', 'pastrami': 'cow', 'corned beef': 'cow', 'honey ham': 'pig', 'nova lox': 'salmon'
}

# problem that some meats capitalized and others not - must convert each val to lowercase
lowercased = data['food'].str.lower()

# map to column animal
data['animal'] = lowercased.map(meat_to_animal)

#combined into one function
data['food'].map(lambda x: meat_to_animal[x.lower()])



0       pig
1       pig
2       pig
3       cow
4       cow
5       pig
6       cow
7       pig
8    salmon
Name: food, dtype: object

### Replacing Values
`replace` provides a simpler and more flexible way to modify a subset of values then the `map` function  
Example:  replace all -999 values with nan  
`data.replace(-999, np.nan)` 

Example: Replace multiple values at once  
`data.replace([-999, -1000], np.nan)`

Different replacement values:  
`data.replace([-999, -1000], [np.nan, 0])`

Pass argument as dict:  
`data.replace({-999: nan, -1000: 0})`


## Renaming Axis Indexes
like values in series axis labels can be transformed by a function or mapping  
can also modify the axes in-place without creating a new data structure



In [30]:
data = pd.DataFrame(np.arange(12).reshape((3,4)), index=['Ohio', 'Colorado', 'Idaho'], columns=['one', 'two', 'three', 'four'])

#like series, axis indexes have a `map` method
transform = lambda x: x[:4].upper()
data.index.map(transform)
# => Index(['OHIO', 'COLO', 'IDAH'], dtype='object')

#assign to index - modifying DF in place
data.index = data.index.map(transform)

#create transformed version of dataset without modifying original => `rename`
data.rename(index=str.title, columns = str.upper)

#rename used with a dict
data.rename(index={'OHIO': 'INDIANA'}, columns={'three': 'peekaboo'})

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLO,4,5,6,7
IDAH,8,9,10,11


## Discretization and Binning
Continuous data is often discretized => seperated into 'bins' for analysis  
use a `cut` function in pandas  
returns a special `Categorical` object  
output describes bins computed by `pandas.cut`  -> array of strings indicating bin name  
internally contains a `categories` array specifying the distinct category names along with a labeling

In [35]:
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]
bins = [18, 25, 35, 60, 100]

cats = pd.cut(ages, bins)
#---> [(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
#---> Length: 12
#---> Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

# labeling for the ages data
cats.codes
# ===> array([0, 0, 0, 1, 0, 0, 2, 1, 3, 2, 2, 1], dtype=int8)

#get value counts for each bucket
pd.value_counts(cats)

#pass custom bin names
group_names = ['Youth', 'young adult', 'middle aged', 'senior']
pd.cut(ages, bins, labels=group_names)

['Youth', 'Youth', 'Youth', 'young adult', 'Youth', ..., 'young adult', 'senior', 'middle aged', 'middle aged', 'young adult']
Length: 12
Categories (4, object): ['Youth' < 'young adult' < 'middle aged' < 'senior']

If you pass an integer number of bins to cut -> compute equal length bins based on min and max values in the data  

In [37]:
data = np.random.rand(20)
cats = pd.cut(data, 4, precision=2)

(0.043, 0.27]    6
(0.27, 0.5]      6
(0.73, 0.95]     5
(0.5, 0.73]      3
dtype: int64

`qcut` bins the data based on sample quantiles  - `cut` will generally not result in each bin having same number of data points  
since `qcut` uses sample quantiles, by definition have roughly equal size bins 


In [39]:
#normally distributed
data = np.random.randn(1000)
#cut into quartiles
cats = pd.qcut(data, 4)

pd.value_counts(cats)

(-3.401, -0.682]      250
(-0.682, -0.00214]    250
(-0.00214, 0.629]     250
(0.629, 3.097]        250
dtype: int64

## Detecting and Filtering Outliers

largely a matter of applying array operations

In [42]:
data = pd.DataFrame(np.random.randn(1000, 4))

#want to find values in one of the columns exceeding 3 in absolute value
col = data[2]
col[np.abs(col) > 3]

#select rows having a value exceeding 3 or -3 use the `any` method on a boolean DF
data[(np.abs(data) > 3).any(1)]

#cap values outside the intervals of -3 and 3
#np.sign(data) gives 1 or -1 depending on pos or neg value
data[np.abs(data)>3] = np.sign(data) *3

 

,0,1,2,3
197,-2.572010,-1.270593,0.777687,3.472184
345,0.817313,-0.738432,-2.439251,-3.088721
502,-0.427646,3.372501,0.321980,-0.325136
564,1.699049,-0.293588,3.031306,1.276238
706,3.257512,1.044002,-0.436217,0.295035
784,-0.956071,-0.242345,-0.424664,-4.099278
789,-2.025761,3.037820,-0.376579,1.487068
942,1.172454,0.424967,3.328213,0.063023
954,-0.698535,-3.294756,-0.009217,0.270611


## Permutation and Random Samlping

Permutation = Randomly Reordering  
`numpy.random.permutation` can randomly reorder a Series or rows of DF  
call it with the length of the axis you want to permute to produce array of integer indicating new ordering  

use sampler array in `iloc` based indexing or the equivalent `take` function  

select a random subset without replacement using `sample` method  

generate a sample with replacement (allow for repeat choices) => pass `replace = True`

In [52]:
df = pd.DataFrame(np.arange(5*4).reshape((5,4)))
sampler = np.random.permutation(5)

# use sampler array in `iloc` based indexing or the equivalent `take` function
df.take(sampler)

#select a random subset without replacement using `sample` method
df.sample(n=3)

#generate a sample with replacement (allow for repeat choices) => pass `replace = True`
choices = pd.Series([5, 7, 9, -4, 2])
draws = choices.sample(n=10, replace=True)


## Computing Indicator / Dummy Variables

Converting categorical variable into a 'dummy' or 'indicator' matrix  
If a col in a DF has `k` distinct values, you would derive a matrix or DF with k cols containing 1's & 0's  
pandas has a `get_dummies` function for this 

might want to add a prefix to the cols in the indicator DF and then merge it with other data  
`prefix='key'` parameter 

if row in DF belongs to multiple categories (pp. 213)
- first extract list of unique genres in the dataset
    - get array of all unique genres
- one way to construct indicator DF is to start with DF of all 0's
    - `zero_matrix = np.zeros((len(movies), len(genres)))
    - `dummies = pd.DataFrame(zero_matrix, columns = genres)`
- now iterate through each movie and set entries in each row of `dummies` to 1
    - use the `dummies.columns` to compute the column indices for each genre
- then use `.iloc` to set values based on these indices
- then combine with movies dataframe


In [56]:
df = pd.DataFrame({'key': ['b', 'b' , 'a', 'c', 'a', 'b'], 'data1': range(6)})

#get dummy vars df
pd.get_dummies(df['key'])

dummies = pd.get_dummies(df['key'], prefix='key')
df_with_dmumy = df[['data1']].join(dummies)
# df_with_dmumy

### RECIPE: combine `get_dummies` with a discretization function like `cut`


In [58]:
np.random.seed(12345)
values = np.random.rand(10)
bins = [0, 0.2, 0.4, 0.6, 0.8, 1]
pd.get_dummies(pd.cut(values, bins))

,"(0.0, 0.2]","(0.2, 0.4]","(0.4, 0.6]","(0.6, 0.8]","(0.8, 1.0]"
0,0,0,0,0,1
1,0,1,0,0,0
2,1,0,0,0,0
3,0,1,0,0,0
4,0,0,1,0,0
5,0,0,1,0,0
6,0,0,0,0,1
7,0,0,0,1,0
8,0,0,0,1,0
9,0,0,0,1,0


## String Manipulation (pp. 215)

In [68]:
#split on comma and strip whitespace
val = 'a,b, guido'
pieces = [x.strip() for x in val.split(',')]
    # => ['a', 'b', 'guido']

# concactenate with join method
'::'.join(pieces) 
    # => 'a::b::guido'

#locating substrings
'guido' in val
#index raises exception if not found while find returns -1
val.index(',')
# => 1
val.find(':')
# => -1

#find number of occurances
val.count(',')
# => 2

#replace
val.replace(',', '::')
# => 'a::b:: guido'



'a::b:: guido'

## Regulare Expressions (pp. 217)

built in `re` module responsible for applying regex to strings

In [71]:
import re
#split with various whitespace
text = 'foo     bar\t baz   \tqux'
re.split('\s+', text)
# => ['foo', 'bar', 'baz', 'qux']

#create a reusable regex func => recommended to save CPU cycles for many uses
regex = re.compile('\s+')
regex.split(text)
# => ['foo', 'bar', 'baz', 'qux']

#list of all matching patterns
regex.findall(text)
# => ['     ', '\t ', '   \t']

# match only matches at the beginning of the string
# search returns a special match object - tells start and end position of the pattern in the string

# sub will return a new string with occurences of the pattern replaced by the new string


['     ', '\t ', '   \t']